# Configuration

In [1]:
# put your email address here
email_address = 'abc_xyz@gmail.com'

# put your openai key here
openai_key = 'sk-TVtpfloxUnesBlbkFJlFTIJmfJPjn311L0eLP3L5FScTUzFa'

# Make sure the Python version is below 3.11

In [2]:
import sys
print(sys.version)

3.10.12 | packaged by Anaconda, Inc. | (main, Jul  5 2023, 19:01:18) [MSC v.1916 64 bit (AMD64)]


# Install and import packages

In [3]:
# for speeding execution, the below lines can be deleted after packages are installed
!pip -q install pywin32
!pip -q install farm-haystack==1.19.0
!pip install -q openai

In [4]:
import os
import openai

# Log in 

In [5]:
import win32com.client as client

In [6]:
outlook = client.Dispatch('Outlook.Application')

In [7]:
namespace = outlook.GetNameSpace('MAPI')

In [8]:
account = namespace.Folders[email_address]

# Get the sent emails

In [9]:
# code for real implementation
sent = account.Folders['Sent Items']
sent_emails = [message for message in sent.Items]

In [10]:
# # for temporary use, should be deleted after real implementation
# sent = account.Folders['Inbox']
# sent_emails = [message for message in sent.Items]

In [ ]:
for s in sent_emails:
    print(str(s.CreationTime))
    print(s.ConversationID)
    print(s.Subject)

# Get the texts of set emails

In [12]:
def remove_links(t):
    import re
    s = re.sub(r"http\S+", "", t)
    return(s)

In [13]:
def remove_rn(t):
    return t.replace('\r\n', '')

In [14]:
def text_proc(t):
    temp = remove_links(t)
    temp2 = remove_rn(temp)
    return temp2

In [15]:
sent_emails_subjects = []
sent_emails2 = []

for s in sent_emails:
    if s.Subject not in sent_emails_subjects:
        sent_emails_subjects.append(s.Subject)
        sent_emails2.append(s)    

In [ ]:
sent_emails_text = ["Subject: "+_.Subject +"\n" +"Body: "+text_proc(_.Body) for _ in sent_emails2]
sent_emails_text

# Save into the folder

In [17]:
from pathlib import Path
doc_dir = str(Path.cwd())
doc_dir

'C:\\Users\\Don\\Desktop\\UC Medical\\Project 4\\test'

In [18]:
path = './sent_emails_text'

# check whether directory already exists
if not os.path.exists(path):
  os.mkdir(path)

In [19]:
cd sent_emails_text

C:\Users\Don\Desktop\UC Medical\Project 4\test\sent_emails_text


In [20]:
# define a write text function to write the above info into texts
def write_text(text, i):
    # write data in a file.
    file = open(str(i)+".txt","w",encoding="utf-8")
    # \n is placed to indicate EOL (End of Line)
    file.write(text)
    file.close() #to change file access modes

In [21]:
for t,s in zip(sent_emails_text, sent_emails2):
    conv_id = str(s.ConversationID)  
    write_text(t,conv_id)

In [22]:
cd ..

C:\Users\Don\Desktop\UC Medical\Project 4\test


# Find all unread emails

In [23]:
inbox = account.Folders['Inbox']

In [24]:
q_mails = []
q_messages = []

for message in inbox.Items:
    # if an unread email contains ? mark, save it into the q_mails list
#     need_reply = message.UnRead == True and "?" in message.Body
    need_reply = message.UnRead == True
    
    if need_reply:
        # get ID, subject and body
        ms_id = message.ConversationID
        ms_subject = message.Subject
        ms_body = message.Body
  
        # combine the above info into one content text
        content = "Subject: " + ms_subject + " Body: " + ms_body
        
        # remove unnecessary info
        content = content.replace("[External Email - Use Caution] ", "")
        
        # save all the above info in one dictionary
        q_mail = {'id':ms_id, 'subject': ms_subject, 'content':content}
            
        # save the mail text and message objects
        q_mails.append(q_mail)        
        q_messages.append(message)

# Find the most relevant previous replies of the unread emails' text

In [25]:
# initiate document store
from haystack.document_stores import InMemoryDocumentStore

document_store = InMemoryDocumentStore(use_bm25=True)

C:\Users\Don\.conda\envs\python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [26]:
# this folder saves my previous replies
from pathlib import Path
doc_dir = str(Path.cwd()) + "\\" + "sent_emails_text"
files_to_index = [doc_dir + "\\" + f for f in os.listdir(doc_dir)]

In [ ]:
# index files from the above files index
from haystack.pipelines.standard_pipelines import TextIndexingPipeline

indexing_pipeline = TextIndexingPipeline(document_store)
indexing_pipeline.run_batch(file_paths=files_to_index)

In [28]:
# processing the texts in the above document store
from haystack import Pipeline
from haystack.nodes import TextConverter, PreProcessor

indexing_pipeline = Pipeline()
text_converter = TextConverter()
preprocessor = PreProcessor(
    clean_whitespace=True,
    clean_header_footer=True,
    clean_empty_lines=True,
    split_by="word",
    split_length=200,
    split_overlap=20,
    split_respect_sentence_boundary=True,
)

In [29]:
indexing_pipeline.add_node(component=text_converter, name="TextConverter", inputs=["File"])
indexing_pipeline.add_node(component=preprocessor, name="PreProcessor", inputs=["TextConverter"])
indexing_pipeline.add_node(component=document_store, name="DocumentStore", inputs=["PreProcessor"])

In [30]:
# create a retriever
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store)

In [31]:
querying_pipeline = Pipeline()
querying_pipeline.add_node(component=retriever, name="Retriever", inputs=["Query"])

In [32]:
# define a get_inquiry function to get inquiries
def get_inquiry(q_mails):   
    
    for i in range(len(q_mails)):
        # get question
        question = q_mails[i]['content']
        
        # get 4 relevant documents
        prediction = querying_pipeline.run(query=question, params={"Retriever": {"top_k": 4}})
        
        # create the background from relevant files
        background = prediction['documents'][0].content + prediction['documents'][1].content + prediction['documents'][2].content +prediction['documents'][3].content
        
        # generate inquiries (to-be-sent to OpenAI)
        inquiry = "What would you reply to the following email inquiry?\n" + question +"\nGiven the background knowledge is: " + background
    
        # append inquiries to q_mails
        q_mails[i]['inquiry'] = inquiry

In [33]:
get_inquiry(q_mails)

In [ ]:
q_mails

# Send the inquiry email and relevant replies to OpenAI for generating responses

In [35]:
# log in OpenAI with the key
openai.api_key = openai_key

In [36]:
def get_draft_text(q_mails):
    for i in range(len(q_mails)):
        
        # get inquiry from q_mails
        inquiry = q_mails[i]['inquiry']
        
        # sent inquiry to OpenAI for generating reponse texts
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                  {"role": "system", "content": "You are a helpful assistant for writing emails."},
                  {"role": "user", "content": inquiry}
              ]
        )
        
        # get draft text
        draft_text = response['choices'][0]['message']['content']

        # save draft text in q_mails       
        q_mails[i]['draft_text'] = draft_text

In [37]:
get_draft_text(q_mails)

In [ ]:
q_mails

# Save the responses in the drafts

In [39]:
# iterate over the q_mail and q_message to generate replies in outlook
for q_mail, q_message in zip(q_mails,q_messages):   
    
    message_id = q_message.ConversationID
        
    # create a reply email object
    reply = q_message.Reply()
    # write the reply body
    reply.Body = q_mail['draft_text']
    # save the reply in the drafts
    reply.Save()      